In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
from time import sleep
from tqdm import tqdm
import glob
import unidecode
import pandas_gbq
from google.oauth2 import service_account
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import requests
import json
import time

In [2]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    ##options.add_argument('--disable-dev-shm-usage')
    ##options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    ##options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    ##options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),options=options)

    #driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver



In [3]:
def get_centros(link_a_seccion):
    driver = driversetup()
    driver.get(link_a_seccion)
    sleep(2)
    html = driver.page_source # get the page source
    soup = BeautifulSoup(html, "html.parser")
    valores = soup.find_all('option', {'ng-value': 'sec.num'})
    centros = []
    for linea in valores:
        centros.append(int(linea.text))
    return centros

In [4]:
def get_mesas(numero_centro):
    driver = driversetup()
    driver.get(f"https://trep.gt/#!/tc4/SEC/e0/m1?secNum={numero_centro}")
    sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    valores = soup.find_all('option', {'ng-value': 'cas.tipoId'})

    mesas_finales = []
    for linea in valores:
        mesas_finales.append(int(linea.text))

    return mesas_finales


# Obtener datos de centros y mesas
En esta sección se guardan en .json los datos de los centros y mesas

In [5]:
"""
centros = get_centros("https://trep.gt/#!/tc4/DIV/e0/m1")
datos_completos = []
for centro in centros:
    mesas = get_mesas(centro)
    datos_completos.append([centro, mesas])

with open("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/intermediate_data/centros_mesas.json", 'w') as f:
    json.dump(datos_completos, f, indent=2)
    """

'\ncentros = get_centros("https://trep.gt/#!/tc4/DIV/e0/m1")\ndatos_completos = []\nfor centro in centros:\n    mesas = get_mesas(centro)\n    datos_completos.append([centro, mesas])\n\nwith open("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/intermediate_data/centros_mesas.json", \'w\') as f:\n    json.dump(datos_completos, f, indent=2)\n    '

In [6]:
with open("/Users/rudikrompich/Git/recopilacion_y_analisis_elecciones_de_guatemala_2023/intermediate_data/centros_mesas.json", 'r') as f:
    data_completa_centros_mesas_numeros = json.load(f)

In [7]:

def get_acta_image(centro, mesa):
    driver = driversetup()
    driver.get(f"https://trep.gt/#!/tc4/CAS/e0/m1/{centro}?casId=M{mesa}")
    sleep(2)
    driver.find_element('xpath','//*[@id="ngview"]/div/main/section[2]/div[1]/div[1]/button').click()
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    images= soup.find_all('img')
    response = requests.get('https://trep.gt'+images[-1]['src'])
    time.sleep(3)

    with open(f'/Users/rudikrompich/dialogos_storage/elecciones_2023/corporacion_municipal/distrito_central/acta_centro_{centro}_mesa_{mesa}.jpg', 'wb') as f:
        f.write(response.content)

# Obtener actas

In [8]:
data_completa_centros_mesas_numeros[69:]

[[70,
  [886, 887, 888, 889, 890, 891, 892, 893, 894, 895, 896, 897, 898, 899, 900]],
 [71,
  [901,
   902,
   903,
   904,
   905,
   906,
   907,
   908,
   909,
   910,
   911,
   912,
   913,
   914,
   915,
   916,
   917]],
 [72,
  [918,
   919,
   920,
   921,
   922,
   923,
   924,
   925,
   926,
   927,
   928,
   929,
   930,
   931,
   932,
   933,
   934,
   935,
   936]],
 [73, [937, 938, 939, 940, 941, 942]],
 [74, [943, 944, 945, 946, 947, 948, 949]],
 [75, [950, 951, 952, 953, 954, 955, 956]],
 [76,
  [957,
   958,
   959,
   960,
   961,
   962,
   963,
   964,
   965,
   966,
   967,
   968,
   969,
   970,
   971,
   972]],
 [77, [973, 974, 975]],
 [78,
  [976,
   977,
   978,
   979,
   980,
   981,
   982,
   983,
   984,
   985,
   986,
   987,
   988,
   989,
   990,
   991,
   992,
   993]],
 [79,
  [994,
   995,
   996,
   997,
   998,
   999,
   1000,
   1001,
   1002,
   1003,
   1004,
   1005,
   1006,
   1007]],
 [80,
  [1008,
   1009,
   1010,
   1011,
 

In [13]:
for dato in data_completa_centros_mesas_numeros[148:]:
    centro = dato[0]
    mesas = dato[1]
    for mesa in mesas:
        get_acta_image(centro, mesa)

In [40]:
get_acta_image(88, 1161)